# Ollama를 활용한 LLM Agent Serving 튜토리얼

### 에이전트 아키텍처

본 튜토리얼에서 구현할 에이전트는 다음 요소들로 구성된다:
- **Pydantic**: 데이터 검증 및 스키마 정의
- **Function Calling & Tool**: 외부 도구 호출 및 실행
- **Memory**: 대화 기록 저장 및 컨텍스트 관리
- **Validation**: 입력 및 출력 데이터 검증
- **Recovery**: 오류 발생 시 복구 메커니즘
- **Feedback**: 실행 결과 피드백 루프

## 1. 환경 설정 및 라이브러리 설치

In [29]:
import json
from typing import List, Dict, Any, Optional, Callable
from datetime import datetime
from pydantic import BaseModel, Field, validator
import requests

## 2. Pydantic 스키마 정의

In [30]:
class ToolParameter(BaseModel):
    """도구 함수의 파라미터를 정의하는 모델이다"""
    name: str = Field(description="파라미터 이름")
    type: str = Field(description="파라미터 타입")
    description: str = Field(description="파라미터 설명")
    required: bool = Field(default=True, description="필수 여부")

class Tool(BaseModel):
    """에이전트가 사용할 수 있는 도구를 정의하는 모델이다"""
    name: str = Field(description="도구 이름")
    description: str = Field(description="도구 기능 설명")
    parameters: List[ToolParameter] = Field(description="도구 파라미터 리스트")
    function: Callable = Field(description="실행할 함수")

class Message(BaseModel):
    """대화 메시지를 표현하는 모델이다"""
    role: str = Field(description="메시지 역할: system, user, assistant")
    content: str = Field(description="메시지 내용")
    timestamp: datetime = Field(default_factory=datetime.now, description="메시지 생성 시간")
    
    @validator('role')
    def validate_role(cls, v):
        """역할이 유효한 값인지 검증한다"""
        allowed_roles = ['system', 'user', 'assistant', 'tool']
        assert v in allowed_roles, f"역할은 {allowed_roles} 중 하나여야 한다"
        return v

class AgentResponse(BaseModel):
    """에이전트의 응답을 표현하는 모델이다"""
    content: str = Field(description="응답 내용")
    tool_calls: Optional[List[Dict[str, Any]]] = Field(default=None, description="호출된 도구 정보")
    is_successful: bool = Field(default=True, description="응답 성공 여부")
    error_message: Optional[str] = Field(default=None, description="오류 메시지")

/tmp/ipykernel_304375/3758584790.py:21: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('role')


## 3. Memory 시스템 구현

In [31]:
class Memory:
    """에이전트의 대화 기록을 관리하는 메모리 시스템이다"""
    
    def __init__(self, max_messages: int = 50):
        """최대 저장 메시지 수를 설정하여 메모리를 초기화한다"""
        self.messages: List[Message] = []
        self.max_messages = max_messages
    
    def add_message(self, role: str, content: str) -> None:
        """새로운 메시지를 메모리에 추가한다"""
        message = Message(role=role, content=content)
        self.messages.append(message)
        
        # 최대 메시지 수를 초과하면 오래된 메시지를 제거한다
        while len(self.messages) > self.max_messages:
            self.messages.pop(0)
    
    def get_messages(self) -> List[Dict[str, str]]:
        """Ollama API 형식으로 메시지 리스트를 반환한다"""
        return [{"role": msg.role, "content": msg.content} for msg in self.messages]
    
    def clear(self) -> None:
        """메모리를 초기화한다"""
        self.messages.clear()
    
    def get_summary(self) -> str:
        """대화 히스토리의 요약 정보를 반환한다"""
        return f"총 {len(self.messages)}개의 메시지가 저장되어 있다"

## 4. Function Calling & Tool 시스템 구현

In [32]:
# 예제 도구 함수들을 정의한다

def calculate(operation: str, x: float, y: float) -> str:
    """두 숫자에 대한 사칙연산을 수행한다"""
    operations = {
        "add": x + y,
        "subtract": x - y,
        "multiply": x * y,
        "divide": x / y
    }
    result = operations.get(operation, "잘못된 연산")
    return f"{operation} 연산 결과: {result}"

def get_weather(city: str) -> str:
    """특정 도시의 날씨 정보를 반환한다 (시뮬레이션)"""
    weather_data = {
        "서울": "맑음, 15도",
        "부산": "흐림, 18도",
        "제주": "비, 20도"
    }
    return weather_data.get(city, f"{city}의 날씨 정보를 찾을 수 없다")

def search_database(query: str) -> str:
    """데이터베이스에서 정보를 검색한다 (시뮬레이션)"""
    return f"'{query}'에 대한 검색 결과: 총 5개의 문서를 찾았다"

In [33]:
class ToolRegistry:
    """도구들을 등록하고 관리하는 레지스트리다"""
    
    def __init__(self):
        """도구 레지스트리를 초기화한다"""
        self.tools: Dict[str, Tool] = {}
    
    def register_tool(self, tool: Tool) -> None:
        """새로운 도구를 레지스트리에 등록한다"""
        self.tools[tool.name] = tool
    
    def get_tool(self, name: str) -> Optional[Tool]:
        """이름으로 도구를 검색한다"""
        return self.tools.get(name)
    
    def execute_tool(self, name: str, **kwargs) -> str:
        """도구를 실행하고 결과를 반환한다"""
        tool = self.get_tool(name)
        result = tool.function(**kwargs)
        return result
    
    def get_tools_description(self) -> str:
        """모든 도구의 설명을 문자열로 반환한다"""
        descriptions = []
        for tool in self.tools.values():
            params_desc = ", ".join([f"{p.name}({p.type})" for p in tool.parameters])
            descriptions.append(f"- {tool.name}({params_desc}): {tool.description}")
        return "\n".join(descriptions)

## 5. Validation 시스템 구현

In [34]:
class Validator:
    """입력 데이터와 도구 호출을 검증하는 클래스다"""
    
    @staticmethod
    def validate_tool_call(tool: Tool, arguments: Dict[str, Any]) -> tuple[bool, Optional[str]]:
        """도구 호출의 인자가 올바른지 검증한다"""
        required_params = [p.name for p in tool.parameters if p.required]
        
        # 필수 파라미터가 모두 제공되었는지 확인한다
        for param in required_params:
            if param not in arguments:
                return False, f"필수 파라미터 '{param}'가 누락되었다"
        
        return True, None
    
    @staticmethod
    def validate_message(message: str) -> tuple[bool, Optional[str]]:
        """메시지 내용이 유효한지 검증한다"""
        if not message or not message.strip():
            return False, "메시지가 비어있다"
        
        if len(message) > 10000:
            return False, "메시지가 너무 길다 (최대 10000자)"
        
        return True, None

## 6. Recovery 시스템 구현

In [35]:
class RecoveryManager:
    """오류 발생 시 복구를 담당하는 관리자다"""
    
    def __init__(self, max_retries: int = 3):
        """최대 재시도 횟수를 설정하여 초기화한다"""
        self.max_retries = max_retries
        self.error_log: List[Dict[str, Any]] = []
    
    def log_error(self, error_type: str, error_message: str, context: Dict[str, Any]) -> None:
        """발생한 오류를 로그에 기록한다"""
        self.error_log.append({
            "timestamp": datetime.now(),
            "error_type": error_type,
            "error_message": error_message,
            "context": context
        })
    
    def get_recovery_prompt(self, error_type: str) -> str:
        """오류 타입에 따른 복구 프롬프트를 생성한다"""
        recovery_prompts = {
            "tool_error": "도구 실행에 실패했다. 다른 방법으로 질문에 답변하라.",
            "validation_error": "입력 검증에 실패했다. 올바른 형식으로 다시 시도하라.",
            "api_error": "API 호출에 실패했다. 간단하게 답변하라."
        }
        return recovery_prompts.get(error_type, "오류가 발생했다. 다시 시도하라.")
    
    def get_error_summary(self) -> str:
        """오류 로그의 요약을 반환한다"""
        return f"총 {len(self.error_log)}개의 오류가 기록되었다"

## 7. Feedback 시스템 구현

In [36]:
class FeedbackCollector:
    """에이전트의 응답에 대한 피드백을 수집하고 분석한다"""
    
    def __init__(self):
        """피드백 컬렉터를 초기화한다"""
        self.feedback_history: List[Dict[str, Any]] = []
    
    def add_feedback(self, query: str, response: str, rating: int, comment: str = "") -> None:
        """새로운 피드백을 추가한다"""
        feedback = {
            "timestamp": datetime.now(),
            "query": query,
            "response": response,
            "rating": rating,
            "comment": comment
        }
        self.feedback_history.append(feedback)
    
    def get_average_rating(self) -> float:
        """평균 평점을 계산한다"""
        if not self.feedback_history:
            return 0.0
        total_rating = sum(f["rating"] for f in self.feedback_history)
        return total_rating / len(self.feedback_history)
    
    def get_feedback_summary(self) -> str:
        """피드백 요약을 반환한다"""
        avg_rating = self.get_average_rating()
        return f"총 {len(self.feedback_history)}개의 피드백, 평균 평점: {avg_rating:.2f}"
    
    def generate_improvement_suggestions(self) -> List[str]:
        """피드백을 기반으로 개선 제안을 생성한다"""
        suggestions = []
        low_rated = [f for f in self.feedback_history if f["rating"] < 3]
        
        if len(low_rated) > len(self.feedback_history) * 0.3:
            suggestions.append("낮은 평점의 응답이 많다. 답변 품질을 개선해야 한다.")
        
        return suggestions

## 8. Ollama 에이전트 통합 구현

In [37]:
class OllamaAgent:
    """Ollama를 사용하는 완전한 에이전트 시스템이다"""
    
    def __init__(self, 
                 model_name: str = "gpt-oss:20b",
                 base_url: str = "http://localhost:11434"):
        """에이전트를 초기화하고 모든 하위 시스템을 설정한다"""
        self.model_name = model_name
        self.base_url = base_url
        self.memory = Memory()
        self.tool_registry = ToolRegistry()
        self.validator = Validator()
        self.recovery_manager = RecoveryManager()
        self.feedback_collector = FeedbackCollector()
        
        # 시스템 프롬프트를 설정한다
        self.system_prompt = self._create_system_prompt()
        self.memory.add_message("system", self.system_prompt)
    
    def _create_system_prompt(self) -> str:
        """도구 정보를 포함한 시스템 프롬프트를 생성한다"""
        tools_desc = self.tool_registry.get_tools_description()
        prompt = f"""당신은 도구를 사용할 수 있는 AI 어시스턴트다.
        
사용 가능한 도구:
{tools_desc}

도구를 사용해야 할 때는 다음 JSON 형식으로 응답하라:
{{
    "tool_name": "도구_이름",
    "arguments": {{"arg1": "value1", "arg2": "value2"}}
}}

도구를 사용하지 않을 때는 일반 텍스트로 답변하라."""
        return prompt
    
    def _call_ollama(self, messages: List[Dict[str, str]]) -> str:
        """Ollama API를 호출하여 응답을 받는다"""
        url = f"{self.base_url}/api/chat"
        payload = {
            "model": self.model_name,
            "messages": messages,
            "stream": False
        }
        
        response = requests.post(url, json=payload)
        result = response.json()
        return result["message"]["content"]
    
    def _parse_tool_call(self, response: str) -> Optional[Dict[str, Any]]:
        """응답에서 도구 호출을 파싱한다"""
        response = response.strip()
        if response.startswith("{") and response.endswith("}"):
            parsed = json.loads(response)
            if "tool_name" in parsed and "arguments" in parsed:
                return parsed
        return None
    
    def run(self, user_input: str, max_iterations: int = 5) -> AgentResponse:
        """사용자 입력을 처리하고 응답을 생성한다"""
        
        # 입력 검증
        is_valid, error_msg = self.validator.validate_message(user_input)
        if not is_valid:
            self.recovery_manager.log_error("validation_error", error_msg, {"input": user_input})
            return AgentResponse(
                content=f"입력 검증 실패: {error_msg}",
                is_successful=False,
                error_message=error_msg
            )
        
        # 사용자 메시지를 메모리에 추가
        self.memory.add_message("user", user_input)
        
        tool_calls_made = []
        
        # 반복적으로 모델을 호출하며 도구 실행
        for iteration in range(max_iterations):
            messages = self.memory.get_messages()
            
            response = self._call_ollama(messages)
            
            # 도구 호출 파싱 시도
            tool_call = self._parse_tool_call(response)
            
            if tool_call:
                tool_name = tool_call["tool_name"]
                arguments = tool_call["arguments"]
                
                tool = self.tool_registry.get_tool(tool_name)
                
                # 도구 호출 검증
                is_valid, error_msg = self.validator.validate_tool_call(tool, arguments)
                if not is_valid:
                    self.recovery_manager.log_error("tool_error", error_msg, 
                                                    {"tool": tool_name, "args": arguments})
                    recovery_prompt = self.recovery_manager.get_recovery_prompt("tool_error")
                    self.memory.add_message("assistant", recovery_prompt)
                    continue
                
                # 도구 실행
                tool_result = self.tool_registry.execute_tool(tool_name, **arguments)
                tool_calls_made.append({
                    "tool": tool_name,
                    "arguments": arguments,
                    "result": tool_result
                })
                
                # 도구 결과를 메모리에 추가
                self.memory.add_message("tool", f"도구 '{tool_name}' 실행 결과: {tool_result}")
            else:
                # 일반 응답인 경우 종료
                self.memory.add_message("assistant", response)
                return AgentResponse(
                    content=response,
                    tool_calls=tool_calls_made if tool_calls_made else None,
                    is_successful=True
                )
        
        # 최대 반복 횟수 도달
        final_response = "최대 반복 횟수에 도달했다. 현재까지의 결과를 반환한다."
        return AgentResponse(
            content=final_response,
            tool_calls=tool_calls_made if tool_calls_made else None,
            is_successful=True
        )
    
    def add_feedback(self, query: str, response: str, rating: int, comment: str = "") -> None:
        """응답에 대한 피드백을 추가한다"""
        self.feedback_collector.add_feedback(query, response, rating, comment)
    
    def get_status(self) -> Dict[str, str]:
        """에이전트의 현재 상태를 반환한다"""
        return {
            "memory": self.memory.get_summary(),
            "errors": self.recovery_manager.get_error_summary(),
            "feedback": self.feedback_collector.get_feedback_summary(),
            "tools": f"{len(self.tool_registry.tools)}개의 도구가 등록되어 있다"
        }

## 9. 도구 등록 및 에이전트 초기화

In [38]:
# 에이전트 초기화
agent = OllamaAgent(model_name="gpt-oss:20b")

# 계산기 도구 등록
calculator_tool = Tool(
    name="calculate",
    description="두 숫자에 대한 사칙연산을 수행한다",
    parameters=[
        ToolParameter(name="operation", type="str", description="연산 종류: add, subtract, multiply, divide"),
        ToolParameter(name="x", type="float", description="첫 번째 숫자"),
        ToolParameter(name="y", type="float", description="두 번째 숫자")
    ],
    function=calculate
)
agent.tool_registry.register_tool(calculator_tool)

# 날씨 도구 등록
weather_tool = Tool(
    name="get_weather",
    description="특정 도시의 날씨 정보를 조회한다",
    parameters=[
        ToolParameter(name="city", type="str", description="도시 이름")
    ],
    function=get_weather
)
agent.tool_registry.register_tool(weather_tool)

# 검색 도구 등록
search_tool = Tool(
    name="search_database",
    description="데이터베이스에서 정보를 검색한다",
    parameters=[
        ToolParameter(name="query", type="str", description="검색 쿼리")
    ],
    function=search_database
)
agent.tool_registry.register_tool(search_tool)

print("Ollama 에이전트가 초기화되었다")
print(f"등록된 도구: {list(agent.tool_registry.tools.keys())}")

Ollama 에이전트가 초기화되었다
등록된 도구: ['calculate', 'get_weather', 'search_database']


## 10. 에이전트 실행 예제

In [39]:
# 예제 1: 일반 대화
print("=== 예제 1: 일반 대화 ===")
response = agent.run("안녕하세요! 당신은 어떤 도움을 줄 수 있나요?")
print(f"응답: {response.content}")
print(f"성공 여부: {response.is_successful}")
print()

=== 예제 1: 일반 대화 ===
응답: 안녕하세요! 저는 다양한 주제에 대해 도움을 드릴 수 있어요. 예를 들면:

- **정보 제공**: 역사, 과학, 문화, 기술 등 궁금한 내용에 대해 설명해 드릴 수 있습니다.  
- **언어 학습**: 한국어, 영어, 다른 언어 학습 팁이나 문법, 어휘를 도와 드립니다.  
- **글쓰기 지원**: 에세이, 보고서, 이메일, 소설 등 문서 작성에 대한 구조, 표현, 교정 등을 도와드립니다.  
- **코딩/프로그래밍**: 파이썬, 자바스크립트, C++ 등 언어에 대한 코드 예시, 디버깅, 개념 설명을 제공할 수 있습니다.  
- **일상 조언**: 식단, 운동, 시간 관리, 인간관계 등 생활 전반에 대한 팁을 드립니다.  
- **창의적 아이디어**: 마케팅 캠페인, 스토리텔링 아이디어, 이벤트 기획 등 창의적인 발상에 도움을 줍니다.  

필요하신 분야를 알려주시면 그에 맞춰 자세히 도와드릴게요!
성공 여부: True



In [40]:
# 예제 2: 도구 사용 - 계산기
print("=== 예제 2: 계산기 도구 사용 ===")
response = agent.run("125와 75를 더한 값을 계산해주세요")
print(f"응답: {response.content}")
print(f"도구 호출: {response.tool_calls}")
print()

=== 예제 2: 계산기 도구 사용 ===
응답: 125 + 75 = **200**
도구 호출: None



In [41]:
# 예제 3: 도구 사용 - 날씨 조회
print("=== 예제 3: 날씨 조회 도구 사용 ===")
response = agent.run("서울의 날씨를 알려주세요")
print(f"응답: {response.content}")
print(f"도구 호출: {response.tool_calls}")
print()

=== 예제 3: 날씨 조회 도구 사용 ===
응답: 
도구 호출: None



In [42]:
# 예제 4: 피드백 추가
print("=== 예제 4: 피드백 시스템 ===")
query = "부산의 날씨를 알려주세요"
response = agent.run(query)
agent.add_feedback(query, response.content, rating=5, comment="매우 정확한 답변이다")
print(f"피드백 요약: {agent.feedback_collector.get_feedback_summary()}")
print()

=== 예제 4: 피드백 시스템 ===
피드백 요약: 총 1개의 피드백, 평균 평점: 5.00



In [43]:
# 예제 5: 에이전트 상태 확인
print("=== 예제 5: 에이전트 상태 ===")
status = agent.get_status()
for key, value in status.items():
    print(f"{key}: {value}")

=== 예제 5: 에이전트 상태 ===
memory: 총 9개의 메시지가 저장되어 있다
errors: 총 0개의 오류가 기록되었다
feedback: 총 1개의 피드백, 평균 평점: 5.00
tools: 3개의 도구가 등록되어 있다


## 11. 메모리 및 컨텍스트 관리 테스트

연속적인 대화를 통해 메모리 시스템이 올바르게 작동하는지 확인한다.

In [44]:
print("=== 메모리 시스템 테스트 ===")

# 첫 번째 대화
response1 = agent.run("제 이름은 김철수입니다")
print(f"응답 1: {response1.content}")

# 두 번째 대화 - 이전 정보 참조
response2 = agent.run("제 이름이 뭐였죠?")
print(f"응답 2: {response2.content}")

print(f"\n현재 메모리: {agent.memory.get_summary()}")

=== 메모리 시스템 테스트 ===
응답 1: 
응답 2: 당신의 이름은 **김철수**입니다!

현재 메모리: 총 13개의 메시지가 저장되어 있다


## 12. Recovery 시스템 테스트

In [45]:
print("=== Recovery 시스템 테스트 ===")

# 빈 메시지 테스트
response = agent.run("")
print(f"빈 메시지 응답: {response.content}")
print(f"오류 메시지: {response.error_message}")

print(f"\n오류 로그: {agent.recovery_manager.get_error_summary()}")

=== Recovery 시스템 테스트 ===
빈 메시지 응답: 입력 검증 실패: 메시지가 비어있다
오류 메시지: 메시지가 비어있다

오류 로그: 총 1개의 오류가 기록되었다
